## Option 2: Academy of Py Analysis:

* Charter schools have better scores (doesn't mean they teach better)
* Worst performing shools spend more money per student
* Math and reading scores are pretty consistent between grades
* Average math score is worst in bigger schools

In [1]:
# Dependencies
import pandas as pd
import numpy as np

# Create a reference the CSV file desired
path_schools = "Resources/schools_complete.csv"
path_students = "Resources/students_complete.csv"

# Read files into a Pandas DataFrame
schools_df = pd.read_csv(path_schools)
students_df = pd.read_csv(path_students)

In [5]:
#rename school column for further merge
schools_df = schools_df.rename(columns={'name':'school'})

In [7]:
#merge two df
summary_df = pd.merge(schools_df, students_df, how='inner', on = 'school')

#remove excess columns
summary_df = summary_df.drop(columns=['size','budget'])
summary_df = summary_df.set_index('Student ID')

**District Summary**

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  

  # **Passing grade set to `70` (change if need to)**

In [8]:
#set passing grade
passing_grade = 70

In [9]:
#setup df
all_summary = schools_df \
                   .groupby('type') \
                   .aggregate( \
                              {'school':pd.Series.nunique, \
                               'size': np.sum, \
                               'budget':np.sum \
                              }
                             )

#rename columns
all_summary.columns = ['Total Schools','Total Students','Total Budget']

#calculate values using groupby
all_summary[['Average Math Score','% Passing Math','Average Reading Score','% Passing Reading']] = \
                  summary_df.groupby('type').agg \
                    ( \
                     {'math_score': \
                      ['mean', lambda x: sum(x > passing_grade) / x.count() * 100], \
                      'reading_score': \
                      ['mean', lambda x: sum(x > passing_grade) / x.count() * 100], \
                     })

#calculate overal passing grade
all_summary['Overall Passing Rate'] = (all_summary['% Passing Math'] + all_summary['% Passing Reading']) / 2

'''
removed format until figure out how not to change underlying values

#format values
all_summary['Total Budget'] = all_summary['Total Budget'].map('${:,.2f}'.format)
all_summary['Total Students'] = all_summary['Total Students'].map('{:,}'.format)
all_summary['Average Math Score'] = all_summary['Average Math Score'].map('{:.2f}'.format)
all_summary['Average Reading Score'] = all_summary['Average Reading Score'].map('{:.2f}'.format)
all_summary['% Passing Math'] = all_summary['% Passing Math'].map('{:.2f}%'.format)
all_summary['% Passing Reading'] = all_summary['% Passing Reading'].map('{:.2f}%'.format)
all_summary['Overall Passing Rate'] = all_summary['Overall Passing Rate'].map('{:.2f}%'.format)
'''

all_summary

,Total Schools,Total Students,Total Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,Overall Passing Rate
type,,,,,,,,
Charter,8,12194,7301505,83.406183,90.282106,83.902821,93.152370,91.717238
District,7,26976,17347923,76.987026,64.305308,80.962485,78.369662,71.337485


**School Summary**

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [10]:
#setup df
school_summary = schools_df \
                   .groupby(['school','type']) \
                   .agg( \
                              {'size': 'sum' , \
                               'budget': 'sum'\
                              }
                             )

#make 'type' index to column
school_summary = school_summary.reset_index('type')

#drop index name
school_summary.index.name = ''

#rename columns
school_summary.columns = ['School Type','Total Students','Total School Budget']

#calculate columns
school_summary['Per Student Budget'] = school_summary['Total School Budget'] / school_summary['Total Students']
school_summary[['Average Math Score','% Passing Math','Average Reading Score','% Passing Reading']] = \
    summary_df.groupby('school').agg \
                    ( \
                     {'math_score': \
                      ['mean', lambda x: sum(x > passing_grade) / x.count()], \
                      'reading_score': \
                      ['mean', lambda x: sum(x > passing_grade) / x.count()], \
                     })
school_summary['Overall Passing Rate'] = (school_summary['% Passing Math'] + school_summary['% Passing Reading'])/2

'''
removed format until figure out how not to change underlying values

#format values
school_summary['Total School Budget'] = school_summary['Total School Budget'].map('${:,.2f}'.format)
school_summary['Per Student Budget'] = school_summary['Per Student Budget'].map('{:,.2f}'.format)
school_summary['Total Students'] = school_summary['Total Students'].map('{:,}'.format)
school_summary['Average Math Score'] = school_summary['Average Math Score'].map('{:.2f}'.format)
school_summary['Average Reading Score'] = school_summary['Average Reading Score'].map('{:.2f}'.format)
school_summary['% Passing Math'] = school_summary['% Passing Math'].map('{:.2%}'.format)
school_summary['% Passing Reading'] = school_summary['% Passing Reading'].map('{:.2%}'.format)
school_summary['Overall Passing Rate'] = school_summary['Overall Passing Rate'].map('{:.2%}'.format)
'''
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,0.646302,81.033963,0.793006,0.719654
Cabrera High School,Charter,1858,1081356,582.0,83.061895,0.895587,83.975780,0.938644,0.917115
Figueroa High School,District,2949,1884411,639.0,76.711767,0.637504,81.158020,0.784334,0.710919
Ford High School,District,2739,1763916,644.0,77.102592,0.657539,80.746258,0.775100,0.716320
Griffin High School,Charter,1468,917500,625.0,83.351499,0.897139,83.816757,0.933924,0.915531
Hernandez High School,District,4635,3022020,652.0,77.289752,0.647465,80.934412,0.781877,0.714671
Holden High School,Charter,427,248087,581.0,83.803279,0.906323,83.814988,0.927400,0.916862
Huang High School,District,2917,1910635,655.0,76.629414,0.633185,81.182722,0.788138,0.710662
Johnson High School,District,4761,3094650,650.0,77.072464,0.638521,80.966394,0.782819,0.710670


**Top Performing Schools (By Passing Rate)**

* Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
#don't need to convert since didn't format the output on previous step
#convert column to float from object
#school_summary['Overall Passing Rate'] = school_summary['Overall Passing Rate'].str.replace('%', '').astype(float)

#create new df with 5 schools
top_schools = school_summary.nlargest(5, 'Overall Passing Rate')

top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Wilson High School,Charter,2283,1319574,578.0,83.274201,0.909330,83.989488,0.932545,0.920937
Pena High School,Charter,962,585858,609.0,83.839917,0.916840,84.044699,0.922037,0.919439
Wright High School,Charter,1800,1049400,583.0,83.682222,0.902778,83.955000,0.934444,0.918611
Cabrera High School,Charter,1858,1081356,582.0,83.061895,0.895587,83.975780,0.938644,0.917115
Holden High School,Charter,427,248087,581.0,83.803279,0.906323,83.814988,0.927400,0.916862


**Bottom Performing Schools (By Passing Rate)**

* Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [12]:
#create new df with 5 schools
bottom_schools = school_summary.nsmallest(5, 'Overall Passing Rate')

bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,% Passing Math,Average Reading Score,% Passing Reading,Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,0.640660,80.744686,0.777444,0.709052
Huang High School,District,2917,1910635,655.0,76.629414,0.633185,81.182722,0.788138,0.710662
Johnson High School,District,4761,3094650,650.0,77.072464,0.638521,80.966394,0.782819,0.710670
Figueroa High School,District,2949,1884411,639.0,76.711767,0.637504,81.158020,0.784334,0.710919
Hernandez High School,District,4635,3022020,652.0,77.289752,0.647465,80.934412,0.781877,0.714671


**Math Scores by Grade**

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [13]:
#group by school and grade calculating mean
math_by_grade = summary_df \
                   .groupby(['school','grade']) \
                   .agg( \
                              {'math_score': 'mean'\
                              }\
                             )
        
#convert level 2 index to columns (pivot)
math_by_grade = math_by_grade.unstack()

#sort column labels alphabetically
math_by_grade = math_by_grade.reindex(sorted(math_by_grade.columns, reverse=True), axis=1)

math_by_grade

math_score                                 
grade                        9th       12th       11th       10th
school                                                           
Bailey High School     77.083676  76.492218  77.515588  76.996772
Cabrera High School    83.094697  83.277487  82.765560  83.154506
Figueroa High School   76.403037  77.151369  76.884344  76.539974
Ford High School       77.361345  76.179963  76.918058  77.672316
Griffin High School    82.044010  83.356164  83.842105  84.229064
Hernandez High School  77.438495  77.186567  77.136029  77.337408
Holden High School     83.787402  82.855422  85.000000  83.429825
Huang High School      77.027251  77.225641  76.446602  75.908735
Johnson High School    77.187857  76.863248  77.491653  76.691117
Pena High School       83.625455  84.121547  84.328125  83.372000
Rodriguez High School  76.859966  77.690748  76.395626  76.612500
Shelton High School    83.420755  83.778976  83.383495  82.917411
Thomas High School     83.590022  83.497041  83.498795  83.087886
Wilson High School     83.085578  83.035794  83.195326  83.724422
Wright High School     83.264706  83.644986  83.836782  84.010288

**Reading Scores by Grade**

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [14]:
#group by school and grade calculating mean
reading_by_grade = summary_df \
                   .groupby(['school','grade']) \
                   .agg( \
                              {'reading_score': 'mean'\
                              }\
                             )
        
#convert level 2 index to columns (pivot)
reading_by_grade = reading_by_grade.unstack()

#sort column labels alphabetically
reading_by_grade = reading_by_grade.reindex(sorted(reading_by_grade.columns, reverse=True), axis=1)

reading_by_grade

reading_score                                 
grade                           9th       12th       11th       10th
school                                                              
Bailey High School        81.303155  80.912451  80.945643  80.907183
Cabrera High School       83.676136  84.287958  83.788382  84.253219
Figueroa High School      81.198598  81.384863  80.640339  81.408912
Ford High School          80.632653  80.662338  80.403642  81.262712
Griffin High School       83.369193  84.013699  84.288089  83.706897
Hernandez High School     80.866860  80.857143  81.396140  80.660147
Holden High School        83.677165  84.698795  83.815534  83.324561
Huang High School         81.290284  80.305983  81.417476  81.512386
Johnson High School       81.260714  81.227564  80.616027  80.773431
Pena High School          83.807273  84.591160  84.335938  83.612000
Rodriguez High School     80.993127  80.376426  80.864811  80.629808
Shelton High School       84.122642  82.781671  84.373786  83.441964
Thomas High School        83.728850  83.831361  83.585542  84.254157
Wilson High School        83.939778  84.317673  83.764608  84.021452
Wright High School        83.833333  84.073171  84.156322  83.812757

**Scores by School Spending**

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [15]:
#no need to convert since cancelled formatting earlier
#convert column to float from object
#school_bins_df['Per Student Budget'] = school_bins_df['Per Student Budget'].str.replace('$', '').astype(float)

#define empty DF with 1 column
school_bins_summary = pd.DataFrame({'Overall Passing Rate' : []})

#assign bins
bins = [0,580,620,640,700]
bins_names = ['<$580','$580-600','$600-620','$>640']

#add Spending Range to new df
school_summary['Spending Ranges (Per Student)'] = pd.cut(school_summary['Per Student Budget'], bins, labels=bins_names)

#calculate results
school_bins_summary[['Average Math Score','% Passing Math','Average Reading Score','% Passing Reading']] = \
    school_summary.groupby('Spending Ranges (Per Student)') \
        .agg( \
             {'Average Math Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()], \
              'Average Reading Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()] \
             })
   
school_bins_summary['Overall Passing Rate'] = \
    (school_bins_summary['% Passing Math'] + school_bins_summary['% Passing Reading'])/2

#rearrange columns
school_bins_summary = school_bins_summary.reindex(columns = ['Average Math Score','Average Reading Score', \
                                              '% Passing Math','% Passing Reading','Overall Passing Rate'])

school_bins_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$580,83.274201,83.989488,1.0,1.0,1.0
$580-600,83.549353,83.903238,1.0,1.0,1.0
$600-620,79.474551,82.120471,1.0,1.0,1.0
$>640,77.023555,80.957446,1.0,1.0,1.0


**Scores by School Size**

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [16]:
#define empty DF with 1 column
school_size_summary = pd.DataFrame({'Overall Passing Rate' : []})

#assign bins
bins_size = [0,1000,3000,5000]
bins_size_names = ['Small(<1000)','Medium(1000-3000)','Large(>3000)']

#add School Size to new df
school_summary['School Size'] = pd.cut(school_summary['Total Students'], bins_size, labels=bins_size_names)

school_size_summary[['Average Math Score','% Passing Math','Average Reading Score','% Passing Reading']] = \
    school_summary.groupby('School Size') \
        .agg( \
             {'Average Math Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()], \
              'Average Reading Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()] \
             })

school_size_summary['Overall Passing Rate'] = \
       (school_size_summary['% Passing Math'] + school_size_summary['% Passing Reading'])/2

#rearrange columns
school_size_summary = school_size_summary.reindex(columns = ['Average Math Score','Average Reading Score', \
                                              '% Passing Math','% Passing Reading','Overall Passing Rate'])

school_size_summary.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,1.0,1.0,1.0
Medium(1000-3000),81.176821,82.933187,1.0,1.0,1.0
Large(>3000),77.063340,80.919864,1.0,1.0,1.0


**Scores by School Type**

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [18]:
type_summary = pd.DataFrame({'Overall Passing Rate' : []})

type_summary[['Average Math Score','% Passing Math','Average Reading Score','% Passing Reading']] = \
    school_summary.groupby('School Type') \
        .agg( \
             {'Average Math Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()], \
              'Average Reading Score': \
                  ['mean', lambda x: sum(x > passing_grade) / x.count()] \
             })
        
type_summary['Overall Passing Rate'] = \
       (type_summary['% Passing Math'] + type_summary['% Passing Reading'])/2

#rearrange columns
type_summary = type_summary.reindex(columns = ['Average Math Score','Average Reading Score', \
                                              '% Passing Math','% Passing Reading','Overall Passing Rate'])
        
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,1.0,1.0,1.0
District,76.956733,80.966636,1.0,1.0,1.0
